In [ ]:
# LIbrary Importing
import threading
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os

In [ ]:
# Variable Assigning
ufa_teams = [
    "hustle", "sol", "glory", "flyers", "union", "summit", "breeze", "mechanix",
    "havoc", "alleycats", "aviators", "radicals", "windchill", "royal", "empire",
    "spiders", "phoenix", "thunderbirds", "steel", "shred", "growlers", "cascades",
    "rush", "bighorns"
]

YEAR = 2025
all_data = []

TIME_LIMIT = 300

In [ ]:
# Fetching Algorithm
def scrape_all():
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    try:
        for team in ufa_teams:
            print(f"Processing team: {team}, year: {YEAR}")
            url = f"https://watchufa.com/stats/player-stats?team={team}&year={YEAR}"
            driver.get(url)
            try:
                table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table")))
                table_html = table.get_attribute("outerHTML")
                tables = pd.read_html(table_html)
                for t in tables:
                    t["Team"] = team
                    t["Year"] = YEAR
                    all_data.append(t)
            except Exception as e:
                print(f"Failed to get data for {team} {YEAR}: {e}")
    finally:
        driver.quit()
scrape_all()

In [ ]:
# Combining and editing
players = pd.concat(all_data, ignore_index=True)

players = players.rename(columns={'+/- ▼':'+/-'})
players['SCR/POS %'] = round(players['SCR'] / players['POS'] * 100, 2)
players['BLK/PP %'] = round(players['BLK']/players['PP'] * 100, 2)
players['PM/PP'] = round(players['+/-']/players['PP'], 2)
players['Y/POS'] = round(players['Y']/players['POS'])
players['HA/POS %'] = round(players['HA']/players['POS'] * 100, 2)

Index(['Player', 'G', 'PP', 'POS', 'SCR', 'AST', 'GLS', 'BLK', '+/-', 'Cmp',
       'Cmp%', 'Y', 'TY', 'RY', 'OEFF', 'HA', 'T', 'S', 'D', 'C', 'Hck',
       'Hck%', 'Pul', 'OPP', 'DPP', 'MP', 'Team', 'Year', 'SCR/POS %',
       'BLK/PP %', 'PM/PP', 'Y/POS', 'HA/POS %'],
      dtype='object')

In [31]:
# Standardizing
stats_col = ['SCR/POS %','BLK/PP %','PM/PP','Y/POS','HA/POS %','Cmp%']
mean = players[stats_col].mean()
std = players[stats_col].std()
players_z = players.copy() 
players_z[stats_col] = round((players[stats_col] - mean) / std,2)

In [32]:
# Saving to Excel
players.to_csv("1 Percentages.csv", index=False)
os.startfile("1 Percentages.csv")
players_z.to_csv("2 ZScores.csv", index=False)
os.startfile("2 ZScores.csv")